In [1]:
import xarray as xr
import os
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
def traverseDir(root):
    for (dirpath, dirnames, filenames) in os.walk(root):
        for file in filenames:
            if file.endswith(('.nc')):
                yield os.path.join(dirpath, file)

In [3]:
root = '/home/pablo/Physics/MasterDS/TFM/TFM/ERA5/ERA5/'

In [4]:
files = np.sort(list(traverseDir(root)))

In [5]:
ds_hourly = xr.open_mfdataset(files[0:120], 
                       concat_dim='time',
                       combine = 'nested',
                       chunks={"time":100})

In [6]:
ds_daily = ds_hourly.resample(time = 'D').sum()

In [7]:
ds_daily['tp']

<xarray.DataArray 'tp' (time: 3652, latitude: 49, longitude: 73)>
dask.array<concatenate, shape=(3652, 49, 73), dtype=float32, chunksize=(1, 49, 73), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 1970-01-01 1970-01-02 ... 1979-12-31
  * longitude  (longitude) float64 -12.0 -11.75 -11.5 -11.25 ... 5.5 5.75 6.0
  * latitude   (latitude) float32 34.0 34.25 34.5 34.75 ... 45.5 45.75 46.0

In [8]:
ds_hourly['tp'][1]

<xarray.DataArray 'tp' (latitude: 49, longitude: 73)>
dask.array<getitem, shape=(49, 73), dtype=float32, chunksize=(49, 73), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float64 -12.0 -11.75 -11.5 -11.25 ... 5.5 5.75 6.0
  * latitude   (latitude) float32 34.0 34.25 34.5 34.75 ... 45.5 45.75 46.0
    time       datetime64[ns] 1970-01-01T01:00:00
Attributes:
    units:      m
    long_name:  Total precipitation

In [ ]:
root1 = '/home/pablo/Physics/MasterDS/TFM/TFM_datascience/pruebas_GAN/datos/netcdf'

In [ ]:
ifiles = np.sort(list(traverseDir(root1)))

In [ ]:
ifiles

In [ ]:
data_raw = xr.open_mfdataset(ifiles, combine='nested', concat_dim='time')

In [ ]:
data_raw

In [ ]:
data_raw = data_raw['__xarray_dataarray_variable__']

In [ ]:
data_raw[3]

In [ ]:
data_raw = data_raw.astype('float32')

In [ ]:
data_raw

In [ ]:
agg = data_raw.resample(time=f'{1}h', label='left').sum(skipna=False)

In [ ]:
agg[2]

In [ ]:
agg.shape

In [ ]:
ds_hourly['tp'].shape

In [ ]:
ds_hourly = ds_hourly['tp'].values
t_per_day = int(24/1)

ntime,ny,nx = ds_hourly.shape
ndays = ntime / t_per_day
assert(ndays.is_integer())
ndays = int(ndays)
reshaped = agg.reshape((ndays,t_per_day,ny,nx))

final = reshaped

In [15]:
### ds_hourly = ds_hourly['tp']
# convert to 32bit
ds_hourly = ds_hourly.astype('float32')


# convert to numpy array
#ds_hourly = ds_hourly.values

# now we want to reshape to (days,tperday,lat,lon)
t_per_day = int(24/1)

ntime,ny,nx = ds_hourly.shape
ndays = ntime / t_per_day
assert(ndays.is_integer())
ndays = int(ndays)
reshaped = ds_hourly.reshape((ndays,t_per_day,ny,nx))

final = reshaped

#np.savez_compressed(f'{outpath}/{startdate}-{enddate}_tres{tres}.npz',data=final)
#np.save(f'{outpath}/{startdate}-{enddate}_tres{tres}', final)

In [16]:
final

array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         ...,
         [ 6.23986125e-06,  8.02241266e-06,  7.13206828e-06, ...,
           0.00000000e+00,  2.67475843e-06,  4.45730984e-06],
         [ 1.78255141e-06,  1.78255141e-06,  8.92207026e-07, ...,
           8.92207026e-07,  2.67475843e-06,  7.13206828e-06],
         [ 8.92207026e-07,  8.92207026e-07,  8.92207026e-07, ...,
           1.78255141e-06,  5.34765422e-06,  9.80496407e-06]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e

In [11]:
ds_hourly

<xarray.DataArray 'tp' (time: 87648, latitude: 49, longitude: 73)>
dask.array<concatenate, shape=(87648, 49, 73), dtype=float32, chunksize=(100, 49, 73), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float64 -12.0 -11.75 -11.5 -11.25 ... 5.5 5.75 6.0
  * latitude   (latitude) float32 34.0 34.25 34.5 34.75 ... 45.5 45.75 46.0
  * time       (time) datetime64[ns] 1970-01-01 ... 1979-12-31T23:00:00
Attributes:
    units:      m
    long_name:  Total precipitation

In [ ]:
# now open all files lazily
# they are automatically chunked per file (thus per day)
#data_raw = xr.open_mfdataset(ifiles, combine='nested', concat_dim='time')
#data_raw = data_raw['__xarray_dataarray_variable__']
# convert to 32bit
#data_raw = data_raw.astype('float32')

# sum to desired timeresolution
#agg = data_raw.resample(time=f'{1}h', label='left').sum(skipna=False)
# convert to numpy array
#agg = agg.values

# now we want to reshape to (days,tperday,lat,lon)
t_per_day = int(24/1)

ntime,ny,nx = agg.shape
ndays = ntime / t_per_day
assert(ndays.is_integer())
ndays = int(ndays)
reshaped = agg.reshape((ndays,t_per_day,ny,nx))

final = reshaped

In [ ]:
final